In [1]:
%load_ext autoreload
%autoreload 2

## Objective

What happens to performance when we add the features from the original dataset?

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))

In [3]:
from transform_for_num_issues_pred import main as transform_dataset

In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

from helper_functions import dummify_cols_and_baselines, make_alphas

In [5]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [6]:
df_transformed = transform_dataset(df_orig)
df_transformed.shape

../preprocessing/transform_for_num_issues_pred.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_subset['NUM_ISSUES'] = 1


(822, 27)

In [ ]:
df_transformed.head(1).T

## Choosing columns

In [7]:
cols_orig_dataset = ['NUM_ISSUES', 'tract_and_block_group']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']

In [8]:
df_col_subset = df_transformed[cols_orig_dataset + cols_census]
df_col_subset.shape

(822, 24)

In [9]:
df = df_col_subset.dropna().drop_duplicates()
df.shape

(558, 24)

## Removing outliers

0303003 is City Hall, which is where issues are assigned to when they don't have a location. Ideally, I would find out which issues truly took place in that block group and filter accordingly.

In [10]:
df = df[df.tract_and_block_group != '0303003'].drop('tract_and_block_group', axis=1)

## Dummify

In [11]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'school', u'housing'], dtype='object')

In [12]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

8_6th_grade is baseline 0 2
rent is baseline 1 2


In [ ]:
df_dummified.shape

In [13]:
df_dummified = df_dummified.drop('race_other', axis=1)

In [14]:
df_dummified.NUM_ISSUES.describe()

count     557.000000
mean      873.858169
std       465.137648
min        50.000000
25%       578.000000
50%       804.000000
75%      1101.000000
max      3007.000000
Name: NUM_ISSUES, dtype: float64

## Running model

In [16]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer


/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('NUM_ISSUES', axis=1), 
    df_dummified.NUM_ISSUES, 
    test_size=0.2, 
    random_state=500
)

In [19]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [ ]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [26]:
# params = {'lassocv__alphas': make_alphas(-5, 5)}
params = {'lassocv__alphas': [[3], [10], [30]]}
# params = {}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=10)
model.fit(X_train, y_train);

In [27]:
pd.DataFrame(model.cv_results_).T.iloc[:5]

,0,1,2
mean_fit_time,0.220216,0.216284,0.20736
mean_score_time,0.0237187,0.0164497,0.00408044
mean_test_score,0.364159,0.383156,0.375614
mean_train_score,0.480241,0.469586,0.43466
param_lassocv__alphas,[3],[10],[30]


Coef values?

In [30]:
coef_values = pd.DataFrame({
    'name': X_train.columns,
    'coef': model.best_estimator_.steps[-1][-1].coef_
})

coef_values['abs_coef'] = pd.np.abs(coef_values.coef)

In [33]:
coef_values[coef_values.coef != 0].sort_values('abs_coef', ascending=False)

,coef,name,abs_coef
14,201.452308,value_std_dev,201.452308
9,137.943432,school_std_dev,137.943432
29,-70.444706,housing_own,70.444706
27,-38.183816,school_21_masters,38.183816
13,29.517070,value,29.517070
5,26.183958,earned_income_per_capita,26.183958
12,-25.854887,bedroom_std_dev,25.854887
10,-14.719475,housing_std_dev,14.719475
15,11.133352,rent,11.133352
26,7.413382,school_20_bachelors,7.413382


In [34]:
model.score(X_test, y_test)

0.39562420523214614

## Conclusion

Adding the additional features improves our model's performance, but we are still in an overfit situation.